# Convert OIDv4 to AutoML Vision

The format accepted by the AutoML Vision is explained [here](https://cloud.google.com/vision/automl/object-detection/docs/csv-format).

In [9]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import os
import warnings
warnings.filterwarnings("ignore")

In [10]:
df = pd.DataFrame(columns=['type','path','label','XMin','YMin','XMax','YMax'])

### Load the Annotations for Images in Train, Test & Validation Sets

In [11]:
df_annot_train = pd.read_csv("F:\\Downloads\\dogs\\OIDv4_ToolKit\\OID\\csv_folder\\train-annotations-bbox.csv")
df_annot_test = pd.read_csv("F:\\Downloads\\dogs\\OIDv4_ToolKit\\OID\\csv_folder\\test-annotations-bbox.csv")
df_annot_validation = pd.read_csv("F:\\Downloads\\dogs\\OIDv4_ToolKit\\OID\\csv_folder\\validation-annotations-bbox.csv")
df_annot_train.head()

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
0,000002b66c9c498e,xclick,/m/01g317,1,0.012500,0.195312,0.148438,0.587500,0,1,0,0,0
1,000002b66c9c498e,xclick,/m/01g317,1,0.025000,0.276563,0.714063,0.948438,0,1,0,0,0
2,000002b66c9c498e,xclick,/m/01g317,1,0.151562,0.310937,0.198437,0.590625,1,0,0,0,0
3,000002b66c9c498e,xclick,/m/01g317,1,0.256250,0.429688,0.651563,0.925000,1,0,0,0,0
4,000002b66c9c498e,xclick,/m/01g317,1,0.257812,0.346875,0.235938,0.385938,1,0,0,0,0


### Retrieve the Respective Label Names for Selected Classes in the Dataset

In [12]:
df_desc = pd.read_csv("F:\\Downloads\\dogs\\OIDv4_ToolKit\\OID\\csv_folder\\class-descriptions-boxable.csv", header=None)
labels = ["Dog", "Bicycle", "Person", "Car", "Fire hydrant", "Motorcycle"]
df_dogcat = df_desc.loc[np.in1d(df_desc[1], labels)]
labels = df_dogcat[0].to_list()
labels

['/m/0199g', '/m/01g317', '/m/01pns0', '/m/04_sv', '/m/0bt9lr', '/m/0k4j']

### Leave Only the Labels We Use in the Annotations DataFrame

In [13]:
# to speed up the search later on
df_annot_train = df_annot_train.loc[np.in1d(df_annot_train["LabelName"], labels)]
df_annot_test = df_annot_test.loc[np.in1d(df_annot_test["LabelName"], labels)]
df_annot_validation = df_annot_validation.loc[np.in1d(df_annot_validation["LabelName"], labels)]

### Create Respective Data Frames

In [14]:
def create_data_frame(path, type = "TRAIN"):
    directory_in_str = path
    directory = os.fsencode(directory_in_str)
    df_new = pd.DataFrame(columns=['type','path','label','XMin','YMin','XMax','YMax'])
    
    if type == "TRAIN"        : df_annot = df_annot_train
    elif type == "TEST"       : df_annot = df_annot_test
    elif type == "VALIDATION" : df_annot = df_annot_validation
    else : raise Exception("Invalid type.")

    for file in os.listdir(directory):
        filename = os.fsdecode(file)

        if filename.endswith(".jpg"):
            name = os.path.splitext(filename)[0]

            df_search = df_annot.loc[np.in1d(df_annot["ImageID"], name)]
            label_type = df_search["LabelName"].iloc[0]

            if   label_type == df_dogcat[0].iloc[0] : label_type = 'Bicycle'
            elif label_type == df_dogcat[0].iloc[1] : label_type = 'Person'
            elif label_type == df_dogcat[0].iloc[2] : label_type = 'Fire hydrant'
            elif label_type == df_dogcat[0].iloc[3] : label_type = 'Motorcycle'
            elif label_type == df_dogcat[0].iloc[4] : label_type = 'Dog'
            elif label_type == df_dogcat[0].iloc[5] : label_type = 'Car'
            else : raise Exception("Sorry, we couldn't find a match for the label name. Please verify the labels.")

            if(len(df_search) > 1):
                for i in range(len(df_search)):
                    #df_search.iloc[i]
                    new_row = {
                        'type'  : type,
                        'path'  : filename,
                        'label' : label_type,
                        'XMin'  : df_search["XMin"].iloc[i],
                        'YMin'  : df_search["YMin"].iloc[i],
                        'XMax'  : df_search["XMax"].iloc[i],
                        'YMax'  : df_search["YMax"].iloc[i],
                    }
                    df_new = df_new.append(new_row, ignore_index=True)
            else:
                new_row = {
                    'type'  : type,
                    'path'  : filename,
                    'label' : label_type,
                    'XMin'  : df_search["XMin"].iloc[0],
                    'YMin'  : df_search["YMin"].iloc[0],
                    'XMax'  : df_search["XMax"].iloc[0],
                    'YMax'  : df_search["YMax"].iloc[0],
                }
                df_new = df_new.append(new_row, ignore_index=True)
                
    return df_new

In [15]:
train_path = "F:\\Downloads\\dogs\\OIDv4_ToolKit\\OID\\Dataset\\train\\Dog_Bicycle_Person_Car_Fire hydrant_Motorcycle\\"
df_train = create_data_frame(train_path, "TRAIN")
df_train

,type,path,label,XMin,YMin,XMax,YMax
0,TRAIN,0000f8aef032941e.jpg,Person,0.000000,0.322702,0.056250,0.639775
1,TRAIN,0000f8aef032941e.jpg,Person,0.035625,0.330206,0.085625,0.590056
2,TRAIN,0000f8aef032941e.jpg,Person,0.278125,0.212946,0.341875,0.623827
3,TRAIN,0000f8aef032941e.jpg,Person,0.290625,0.268293,0.345625,0.775797
4,TRAIN,0000f8aef032941e.jpg,Person,0.316250,0.268293,0.470000,0.916510
...,...,...,...,...,...,...,...
16758,TRAIN,ffe223986b19bec3.jpg,Car,0.511438,0.651961,0.808824,0.864379
16759,TRAIN,ffe91d5b90980783.jpg,Person,0.064375,0.000000,0.324375,0.369748
16760,TRAIN,fffb6bce3b6e9498.jpg,Car,0.062500,0.541250,0.427500,0.703125
16761,TRAIN,fffb6bce3b6e9498.jpg,Car,0.065000,0.549375,0.196667,0.602500


In [16]:
test_path = "F:\\Downloads\\dogs\\OIDv4_ToolKit\\OID\\Dataset\\test\\Dog_Bicycle_Person_Car_Fire hydrant_Motorcycle\\"
df_test = create_data_frame(test_path, "TEST")
df_test

,type,path,label,XMin,YMin,XMax,YMax
0,TEST,0003d84e0165d630.jpg,Person,0.225808,0.199143,0.295419,0.433856
1,TEST,0003d84e0165d630.jpg,Person,0.230405,0.027487,0.293449,0.265703
2,TEST,0003d84e0165d630.jpg,Person,0.442522,0.034493,0.512133,0.297232
3,TEST,0003d84e0165d630.jpg,Person,0.735415,0.014350,0.837862,0.384812
4,TEST,0013a0927e6bbefc.jpg,Person,0.254002,0.290806,0.464677,0.616060
...,...,...,...,...,...,...,...
5232,TEST,ff984272d363823a.jpg,Bicycle,0.029181,0.101846,0.629715,0.719195
5233,TEST,ff984272d363823a.jpg,Bicycle,0.472768,0.179821,0.657511,0.382431
5234,TEST,ff984272d363823a.jpg,Bicycle,0.543147,0.251441,0.600957,0.355102
5235,TEST,ff984272d363823a.jpg,Bicycle,0.605984,0.249557,0.676363,0.396566


In [17]:
validation_path = "F:\\Downloads\\dogs\\OIDv4_ToolKit\\OID\\Dataset\\validation\\Dog_Bicycle_Person_Car_Fire hydrant_Motorcycle\\"
df_validation = create_data_frame(validation_path, "VALIDATION")
df_validation

,type,path,label,XMin,YMin,XMax,YMax
0,VALIDATION,0007d6cf88afaa4a.jpg,Dog,0.173566,0.216627,0.902569,0.941628
1,VALIDATION,000c4d66ce89aa69.jpg,Dog,0.220773,0.083870,0.865219,0.924838
2,VALIDATION,001a995c1e25d892.jpg,Motorcycle,0.102043,0.114239,0.877043,0.945402
3,VALIDATION,0022d0ab1e1347ab.jpg,Dog,0.048164,0.170309,1.000000,0.999717
4,VALIDATION,0022e32008e479cb.jpg,Car,0.000001,0.000000,0.999991,1.000000
...,...,...,...,...,...,...,...
3919,VALIDATION,ff69d090c735eccb.jpg,Car,0.646926,0.000000,0.999042,0.355109
3920,VALIDATION,ff7b4cc8ca9b6592.jpg,Person,0.317006,0.102351,0.795478,0.998400
3921,VALIDATION,ff9a9066c7b74983.jpg,Dog,0.000000,0.340499,0.485855,0.862865
3922,VALIDATION,ff9a9066c7b74983.jpg,Dog,0.382260,0.272008,0.863983,0.784938


In [18]:
# add path to images
df_train['path'] = train_path + df_train['path'].astype(str)
df_validation['path'] = validation_path + df_validation['path'].astype(str)
df_test['path'] = test_path + df_test['path'].astype(str)

# join all frames
frames = [df_train, df_validation, df_test]
df_final = pd.concat(frames)
df_final

,type,path,label,XMin,YMin,XMax,YMax
0,TRAIN,F:\Downloads\dogs\OIDv4_ToolKit\OID\Dataset\tr...,Person,0.000000,0.322702,0.056250,0.639775
1,TRAIN,F:\Downloads\dogs\OIDv4_ToolKit\OID\Dataset\tr...,Person,0.035625,0.330206,0.085625,0.590056
2,TRAIN,F:\Downloads\dogs\OIDv4_ToolKit\OID\Dataset\tr...,Person,0.278125,0.212946,0.341875,0.623827
3,TRAIN,F:\Downloads\dogs\OIDv4_ToolKit\OID\Dataset\tr...,Person,0.290625,0.268293,0.345625,0.775797
4,TRAIN,F:\Downloads\dogs\OIDv4_ToolKit\OID\Dataset\tr...,Person,0.316250,0.268293,0.470000,0.916510
...,...,...,...,...,...,...,...
5232,TEST,F:\Downloads\dogs\OIDv4_ToolKit\OID\Dataset\te...,Bicycle,0.029181,0.101846,0.629715,0.719195
5233,TEST,F:\Downloads\dogs\OIDv4_ToolKit\OID\Dataset\te...,Bicycle,0.472768,0.179821,0.657511,0.382431
5234,TEST,F:\Downloads\dogs\OIDv4_ToolKit\OID\Dataset\te...,Bicycle,0.543147,0.251441,0.600957,0.355102
5235,TEST,F:\Downloads\dogs\OIDv4_ToolKit\OID\Dataset\te...,Bicycle,0.605984,0.249557,0.676363,0.396566


In [19]:
#x_relative_min,y_relative_min,x_relative_max,y_relative_min,x_relative_max,y_relative_max,x_relative_min,y_relative_max

# rearrange columns
df_final = df_final[["type", "path", "label", "XMin", "YMin", "XMax", "YMin", "XMax", "YMax", "XMin", "YMax"]]
df_final.head()

,type,path,label,XMin,YMin,XMax,YMin,XMax,YMax,XMin,YMax
0,TRAIN,F:\Downloads\dogs\OIDv4_ToolKit\OID\Dataset\tr...,Person,0.000000,0.322702,0.056250,0.322702,0.056250,0.639775,0.000000,0.639775
1,TRAIN,F:\Downloads\dogs\OIDv4_ToolKit\OID\Dataset\tr...,Person,0.035625,0.330206,0.085625,0.330206,0.085625,0.590056,0.035625,0.590056
2,TRAIN,F:\Downloads\dogs\OIDv4_ToolKit\OID\Dataset\tr...,Person,0.278125,0.212946,0.341875,0.212946,0.341875,0.623827,0.278125,0.623827
3,TRAIN,F:\Downloads\dogs\OIDv4_ToolKit\OID\Dataset\tr...,Person,0.290625,0.268293,0.345625,0.268293,0.345625,0.775797,0.290625,0.775797
4,TRAIN,F:\Downloads\dogs\OIDv4_ToolKit\OID\Dataset\tr...,Person,0.316250,0.268293,0.470000,0.268293,0.470000,0.916510,0.316250,0.916510


In [20]:
df_final.label.value_counts()

Person          10403
Bicycle          5915
Car              4083
Motorcycle       2689
Dog              2310
Fire hydrant      524
Name: label, dtype: int64

There is a clear imbalance in our class, with majority being Person class and minority a Fire hydrant. This is a problem we need to address

<hr/>

In [21]:
df_final.type.value_counts()

TRAIN         16763
TEST           5237
VALIDATION     3924
Name: type, dtype: int64

In [22]:
#df_annot[df_annot.ImageID == "00a2f363f2028056"]

NameError: name 'df_annot' is not defined

In [23]:
df_final.loc[df_final.path.str.contains("00a2f363f2028056")].sort_values(by = "type", ascending=False)

,type,path,label,XMin,YMin,XMax,YMin,XMax,YMax,XMin,YMax


# Convert PascalVOC to AutoML Vision

In [83]:
df_pascal = pd.DataFrame(columns=['type','path','label','XMin','YMin','XMax','YMax'])
df_pascal.head()

path_to_images = "E:/Images/SideHelper/"

directory_in_str = "E:/Images/SideHelper/xml/"
directory = os.fsencode(directory_in_str)

for file in os.listdir(directory):
    filename = os.fsdecode(file)

    if filename.endswith(".xml"):
        name = os.path.splitext(filename)[0]
        path = directory_in_str + filename
        #print(name)
        
        # X divided by the width
        # Y divided by the height
        # how to calculate normalized coordinates: https://stackoverflow.com/questions/48915003/get-the-bounding-box-coordinates-in-the-tensorflow-object-detection-api-tutorial

        tree = ET.parse(path)
        root = tree.getroot()
        image_name = path_to_images + root.find('filename').text
        
        for size in root.findall('size'):
            width = int(size.find('width').text)
            height = int(size.find('height').text)
            #print(width, height)
            
        for object in root.findall('object'):
            label_type = object.find('name').text
            for bndbox in object.findall('bndbox'):
                xmin = int(bndbox.find('xmin').text) / width
                xmax = int(bndbox.find('xmax').text) / width
                ymin = int(bndbox.find('ymin').text) / height
                ymax = int(bndbox.find('ymax').text) / height
                #print(xmin, xmax, ymin, ymax)
                
                new_row = {
                 'type':"TRAIN",
                 'path':image_name,
                 'label':label_type,
                 'XMin':xmin,
                 'YMin':ymin,
                 'XMax':xmax,
                 'YMax':ymax,
                 }

                df_pascal = df_pascal.append(new_row, ignore_index=True)

df_pascal

,type,path,label,XMin,YMin,XMax,YMax
0,TRAIN,E:/Images/SideHelper/bicycle0.jpg,Bicycle,0.409273,0.285587,0.546698,0.581406
1,TRAIN,E:/Images/SideHelper/bicycle0.jpg,Bicycle,0.584390,0.337189,0.721147,0.806050
2,TRAIN,E:/Images/SideHelper/bicycle0.jpg,Person,0.591394,0.126335,0.755170,0.582740
3,TRAIN,E:/Images/SideHelper/bicycle0.jpg,Person,0.432622,0.117883,0.548699,0.436833
4,TRAIN,E:/Images/SideHelper/bicycle0.jpg,Person,0.271848,0.200623,0.311541,0.383897
...,...,...,...,...,...,...,...
486,TRAIN,E:/Images/SideHelper/tree3.jpg,Tree,0.283784,0.364177,0.358722,0.556876
487,TRAIN,E:/Images/SideHelper/tree4.jpg,Tree,0.632721,0.254296,0.676127,0.938144
488,TRAIN,E:/Images/SideHelper/tree4.jpg,Tree,0.383973,0.393471,0.419032,0.718213
489,TRAIN,E:/Images/SideHelper/tree5.jpg,Tree,0.302000,0.311377,0.358000,0.940120


In [48]:
df_pascal.path.iloc[2]

'F:/Downloads/dogs/google-images/google-images-download/google_images_download/downloads/sidewalk cone/sidewalk-cone1.jpg'

In [84]:
df_pascal.label.value_counts()

Cone            131
Person           78
Pole             56
Fire hydrant     42
Dog              32
Pothole          30
Tree             28
Bicycle          27
Trash can        27
Car              25
Motorcycle       15
Name: label, dtype: int64

In [50]:
df_pascal[df_pascal["path"] == "sidewalk-cone28.jpg"]

,type,path,label,XMin,YMin,XMax,YMax


In [85]:
df_count = (int)(len(df_pascal.index) * .10)
df_sample_test = df_pascal.sample(df_count)
df_sample_test.type = "TEST"
df_pascal.update(df_sample_test)
test_list = df_sample_test.index.to_list()
df_pascal.head()

,type,path,label,XMin,YMin,XMax,YMax
0,TRAIN,E:/Images/SideHelper/bicycle0.jpg,Bicycle,0.409273,0.285587,0.546698,0.581406
1,TRAIN,E:/Images/SideHelper/bicycle0.jpg,Bicycle,0.584390,0.337189,0.721147,0.806050
2,TRAIN,E:/Images/SideHelper/bicycle0.jpg,Person,0.591394,0.126335,0.755170,0.582740
3,TRAIN,E:/Images/SideHelper/bicycle0.jpg,Person,0.432622,0.117883,0.548699,0.436833
4,TRAIN,E:/Images/SideHelper/bicycle0.jpg,Person,0.271848,0.200623,0.311541,0.383897


In [86]:
df_count = (int)(len(df_pascal.index) * .10)
df3 = df_pascal[~df_pascal.index.isin(test_list)].copy()
df_sample_validation = df3.sample(df_count)
df_sample_validation.type = "VALIDATION"
df_pascal.update(df_sample_validation)
validation_list = df_sample_validation.index.to_list()
df_pascal.head()

,type,path,label,XMin,YMin,XMax,YMax
0,TRAIN,E:/Images/SideHelper/bicycle0.jpg,Bicycle,0.409273,0.285587,0.546698,0.581406
1,TRAIN,E:/Images/SideHelper/bicycle0.jpg,Bicycle,0.584390,0.337189,0.721147,0.806050
2,VALIDATION,E:/Images/SideHelper/bicycle0.jpg,Person,0.591394,0.126335,0.755170,0.582740
3,VALIDATION,E:/Images/SideHelper/bicycle0.jpg,Person,0.432622,0.117883,0.548699,0.436833
4,VALIDATION,E:/Images/SideHelper/bicycle0.jpg,Person,0.271848,0.200623,0.311541,0.383897


In [87]:
df_pascal.type.value_counts()

TRAIN         393
VALIDATION     49
TEST           49
Name: type, dtype: int64

In [88]:
#x_relative_min,y_relative_min,x_relative_max,y_relative_min,x_relative_max,y_relative_max,x_relative_min,y_relative_max
#xmin            ymin          xmax    ymin          xmax         ymax    xmin             ymax
#0.41964287	0.08240224	0.64285713	0.08240224	0.64285713	0.6424581	0.41964287	0.6424581
#0.60491073	0.09078212	0.83035713	0.09078212	0.83035713	0.8687151	0.60491073	0.8687151
#0.14285715	0.025139665	0.44419643	0.025139665	0.44419643	0.44972068	0.14285715	0.44972068


# rearrange columns
df_pascal = df_pascal[["type", "path", "label", "XMin", "YMin", "XMax", "YMin", "XMax", "YMax", "XMin", "YMax"]]
df_pascal.head()

,type,path,label,XMin,YMin,XMax,YMin,XMax,YMax,XMin,YMax
0,TRAIN,E:/Images/SideHelper/bicycle0.jpg,Bicycle,0.409273,0.285587,0.546698,0.285587,0.546698,0.581406,0.409273,0.581406
1,TRAIN,E:/Images/SideHelper/bicycle0.jpg,Bicycle,0.584390,0.337189,0.721147,0.337189,0.721147,0.806050,0.584390,0.806050
2,VALIDATION,E:/Images/SideHelper/bicycle0.jpg,Person,0.591394,0.126335,0.755170,0.126335,0.755170,0.582740,0.591394,0.582740
3,VALIDATION,E:/Images/SideHelper/bicycle0.jpg,Person,0.432622,0.117883,0.548699,0.117883,0.548699,0.436833,0.432622,0.436833
4,VALIDATION,E:/Images/SideHelper/bicycle0.jpg,Person,0.271848,0.200623,0.311541,0.200623,0.311541,0.383897,0.271848,0.383897


In [55]:
# to make a sample % a test, validation set from training #

#df_count = (int)(len(df2.index) * .15)
#df3 = df2[~df2.index.isin(test_list)].copy()
#df_sample_valid = df3.sample(df_count)
#df_sample_valid.type = "VALIDATION"
#df2.update(df_sample_valid)
#validation_list = df_sample_valid.index.to_list()
#df2.head()

### Join OIDv4 & Pascal DataFrames

In [56]:
df_sidehelper = pd.concat([df_final, df_pascal], ignore_index=True, sort=False)

In [89]:
#df_sidehelper.to_csv("F:/Downloads/dogs/OIDv4_ToolKit/OID/sidehelper.csv", index=False, header=False)
df_pascal.to_csv("F:/Downloads/dogs/OIDv4_ToolKit/OID/sidehelper2.csv", index=False, header=False)

In [58]:
df_sidehelper.label.value_counts()

Person          10403
Bicycle          5915
Car              4083
Motorcycle       2689
Dog              2310
Fire hydrant      525
Cone              106
Name: label, dtype: int64

In [59]:
df_sidehelper.type.value_counts()
df_sidehelper.path.nunique()

9513

In [2]:
import os
import cv2
def check_images(s_dir, ext_list):
    bad_images=[]
    bad_ext=[]
    s_list= os.listdir(s_dir)
    for klass in s_list:
        klass_path=os.path.join (s_dir, klass)
        print ('processing class directory ', klass)
        if os.path.isdir(klass_path):
            file_list=os.listdir(klass_path)
            for f in file_list:               
                f_path=os.path.join (klass_path,f)
                index=f.rfind('.')
                ext=f[index+1:].lower()
                if ext not in ext_list:
                    print('file ', f_path, ' has an invalid extension ', ext)
                    bad_ext.append(f_path)
                if os.path.isfile(f_path):
                    try:
                        img=cv2.imread(f_path)
                        shape=img.shape
                    except:
                        print('file ', f_path, ' is not a valid image file')
                        bad_images.append(f_path)
                else:
                    print('*** fatal error, you a sub directory ', f, ' in class directory ', klass)
        else:
            print ('*** WARNING*** you have files in ', s_dir, ' it should only contain sub directories')
    return bad_images, bad_ext

source_dir =r'F:\Downloads\dogs\OIDv4_ToolKit\OID\Dataset\validation'
good_exts=['jpg', 'jpeg' ] # list of acceptable extensions
bad_file_list, bad_ext_list=check_images(source_dir, good_exts)
if len(bad_file_list) !=0:
    print('improper image files are listed below')
    for i in range (len(bad_file_list)):
        print (bad_file_list[i])
else:
    print(' no improper image files were found')

processing class directory  Dog_Bicycle_Person_Car_Fire hydrant_Motorcycle
 no improper image files were found


### To Convert Invalid Images to the Correct Format

In [74]:
#credit: https://discuss.tensorflow.org/t/valueerror-image-format-not-jpeg/2143/12

import glob
from PIL import Image
files = glob.glob(r"F:\Downloads\dogs\google-images\google-images-download\google_images_download\downloads\*\*")
print(len(files))
for file in files:
    if ".jpg" in file:
        image = Image.open(file)
    if image.format not in ["JPG", "JPEG"]:
        print (file)
        image.convert("RGB").save(file, "JPEG")

732
F:\Downloads\dogs\google-images\google-images-download\google_images_download\downloads\car sidewalk\car94.jpg
F:\Downloads\dogs\google-images\google-images-download\google_images_download\downloads\dog sidewalk\dog1.jpg
F:\Downloads\dogs\google-images\google-images-download\google_images_download\downloads\dog sidewalk\dog3.jpg
F:\Downloads\dogs\google-images\google-images-download\google_images_download\downloads\dog sidewalk\dog43.jpg
F:\Downloads\dogs\google-images\google-images-download\google_images_download\downloads\dog sidewalk\dog7.jpg
F:\Downloads\dogs\google-images\google-images-download\google_images_download\downloads\fire hydrant sidewalk\fire-hydrant18.jpg
F:\Downloads\dogs\google-images\google-images-download\google_images_download\downloads\fire hydrant sidewalk\fire-hydrant23.jpg
F:\Downloads\dogs\google-images\google-images-download\google_images_download\downloads\fire hydrant sidewalk\fire-hydrant28.jpg
F:\Downloads\dogs\google-images\google-images-download\go

### Rename Files

In [79]:
import os
path = 'F:\\Downloads\\dogs\\google-images\\google-images-download\\google_images_download\\downloads\\tree sidewalk\\'
path = r'E:\Images\SideHelper\pothole'
files = os.listdir(path)


for index, file in enumerate(files):
    os.rename(os.path.join(path, file), os.path.join(path, ''.join(['pothole' + str(index), '.jpg'])))

### Convert XML to CSV

In [ ]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET

def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def main():
    image_path = os.path.join(os.getcwd(), 'annotations')
    xml_df = xml_to_csv(image_path)
    xml_df.to_csv('raccoon_labels.csv', index=None)
    print('Successfully converted xml to csv.')


main()